In [21]:
import os
os.chdir(r'C:\Users\91996\Text-Summarizer')
%pwd

'C:\\Users\\91996\\Text-Summarizer'

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [23]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
        )

        return model_evaluation_config

In [25]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm

In [26]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the ddata dataset into smaller batches
        to process in parallel. Yield successive batch-sized chunks from list_of_elements"""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i: i+batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer, 
                                    batch_size = 8, device = 'cuda' if torch.cuda.is_available() else 'cpu',
                                    column_text = 'article',
                                     column_summary = 'highlights'):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total = len(article_batches)
        ):
            inputs = tokenizer(article_batch, max_length=1024,
                               truncation = True, padding = "max_length",
                               return_tensors = 'pt')
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_masl = inputs["attention_mask"].to(device),
                                       length_penalty = 0.8, num__beams = 8, max_length = 128)
            
            '''parameter for length penalty ensures that model does not generate sequence that are too long'''

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                                  clean_up_tokenization_spaces = True)
                                                  for s in summaries]
            
            decode_summaries = [f.replace("", " ") for f in decoded_summaries]

            metric.add_batch(predictions = decoded_summaries, references = target_batch)

        score = metric.compute()
        return score
    
    def evaluate(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rougeL"]

        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10]. 
            rouge_metric, model_pegasus, tokenizer,
            batch_size = 2, column_text= 'article', column_summary = 'highlights'
        )
        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index = ['pegasus'])
        df.to_csv(self.config.metric_file_name, index = False)


In [27]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.evaluate()

except Exception as e:
    raise e

[2025-04-13 07:21:26,735: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-13 07:21:26,749: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-13 07:21:26,752: INFO: common: created directory at: artifacts]
[2025-04-13 07:21:26,755: INFO: common: created directory at: artifacts/model_evaluation]


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'artifacts/model_trainer/tokenizer'. Use `repo_type` argument if needed.